In [3]:
from Graph import Graph

# initialize graph
graph = Graph()

In [5]:
# retrieve active_player_histories-- this will be used to create connections
import pandas as pd
from datetime import datetime

# initialize players dict for O(1) lookup of player information
players_csv = pd.read_csv("active_players_nba_2023.csv")
players = {}

for row in players_csv.iterrows():
    content = row[1]
    players[content["id"]] = content["name"]


player_trades = pd.read_csv("active_player_histories_nba.csv")
player_trades.fillna(datetime.today().strftime('%Y-%m-%d'), inplace=True)
player_trades.sort_values("end", inplace=True, ascending=False)
player_trades.head(5)
# insertion will be done by generating current teams and replaying trades from present -> past
# the sorting allows this



,Unnamed: 0,player_id,player_name,team,start,end
0,0,817057,DeMar DeRozan,Chicago Bulls,2021-08-02,2023-04-30
697,0,817022,Clint Capela,Atlanta Hawks,2020-02-04,2023-04-30
700,1181233,1181233,AJ Griffin,Atlanta Hawks,2023-04-30,2023-04-30
701,0,1133787,Jalen Johnson,Atlanta Hawks,2021-07-28,2023-04-30
702,0,987933,Jarrett Culver,Memphis Grizzlies,2021-08-16,2023-04-30


In [6]:
import heapq
import datetime

current_date = None
teams = {}
current_team = {}

# remove at start_date
next_date = []
remove_date_map = {}

# this code will break at 9001
BIG_DATE = datetime.datetime.strptime("9001-12-25", "%Y-%m-%d").date()

def remove_from_team(player):
    curr_team = current_team.get(player)
    if curr_team != None: 
        del current_team[player]
        teams[curr_team].remove(player)

def add_to_team(player, team, start_date):
    # init team if not initialized
    teams[team] = teams.get(team, set())

    # remove player from their current team
    remove_from_team(player)

    # set player current team
    current_team[player] = team

    if remove_date_map.get(start_date) == None:
        remove_date_map[start_date] = []
    
    start_date_dt = datetime.datetime.strptime(start_date, "%Y-%m-%d").date()
    heapq.heappush(next_date, ((BIG_DATE - start_date_dt).days, start_date))
    remove_date_map[start_date].append(player)

    # add player to team
    teams[team].add(player)

def connect_players(players):
    players = list(players) # convert to an array, so we can index[0]

    for i in range(len(players)):
        curr_player = players[i]
        for j in range(i + 1, len(players)):
            connect_player = players[j]
            graph.addNeighbor(curr_player, connect_player)

# this connects all teams at their current state
def flush_teams():
    global current_date
    for _, players in teams.items():
        connect_players(players)
    current_date = None

# iterate through all player trades
for row in player_trades.iterrows():
    content = row[1]
    date = content["end"]

    if current_date != date:
         flush_teams()

    if current_date == None:
        current_date = date

        # remove all players based on their start date
        while len(next_date) > 0:
            date_to_compare = next_date[0][1]
            if date_to_compare < current_date:
                break

            heapq.heappop(next_date)
            for player in remove_date_map[date_to_compare]:
                remove_from_team(player)
    
    add_to_team(content["player_id"], content["team"], content["start"])



In [8]:
import networkx as nx 
networkG = nx.Graph(graph.connections)
pos = nx.spring_layout(networkG, k=.2)

output = {
    "edges": [],
    "nodes": []
}

max_edges = 0

# fill edges
for source, targets in graph.connections.items():
    max_edges = max(max_edges, len(targets))
    for target in targets:
        output["edges"].append({
            "source": source,
            "target": target,
        })


largest_node_size = 7.5


# fill nodess
for key, value in pos.items():
    output["nodes"].append({
        "key": key,
        "attributes": {
            "x": value[0],
            "y": value[1],
            "size": len(graph.connections[key])/max_edges * largest_node_size,
            "label": players[key]
        }
    })

import json

with open("../data.json", "w") as outfile:
    json.dump(output, outfile)


In [ ]:
for id1 in graph.connections:
    for id2 in graph.connections:
        if id1==id2:
            continue
        if len(graph.bfs(id1, id2)) > 3:
            output = []
            for player in graph.bfs(id1, id2):
                matched_row = player_trades[player_trades['player_id'] == player]
                matched_name = matched_row['player_name'].values[0]
                output.append(matched_name)
            print(output)

In [10]:
graph.bfs(1178199, 817181)

[987099, 817147, 817307]